# koalpaca : 한국어 버전의 alpca 모델

alpaca : llama라는 모델을 instruction fine-tuning 해서 사람의 지시를 잘 따르는 모델

llama라는 모델은 LLaMA-13B의 경우 GPT-3보다 10배이상 작음에도 불구하고 대부분의 평가서 GPT-3보다 우수한 성능을 보이며, 더 나아가 LLaMA-65B의 경우 대부분의 벤치마크에서 Chinchilla, Gopher, GPT-3, PaLM와 유사하거나 더 뛰어난 결과를 보이는 훌륭한 모델입니다.

그러면 지금부터 koalpaca로 생성 모델을 한번 경험해보도록 하겠습니다!

In [12]:
# GPU 켜져 있는지 확인

!nvidia-smi

Wed Aug  7 06:28:02 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P0              30W /  70W |   7943MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [13]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [14]:
!pip install -q -U git+https://github.com/huggingface/accelerate.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [15]:
!pip install -q datasets

In [16]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# peft
peft는 parameter efficient fine-tuning의 약자로, 큰 모델을 코랩에서 돌릴 수 있도록 쪼개서 만든 것입니다.

오래 걸립니다!

In [17]:
from peft import PeftModel, PeftConfig

In [18]:
# 효율적인 추론을 위해 사전 학습되고 미세 조정된 언어 모델을 준비하는 과정!!
# 양자화를 통해 메모리 사용을 줄이는 데 중점을 둔다.

peft_model_id = "beomi/qlora-koalpaca-polyglot-12.8b-50step" # 파라미터 효율적인 미세조정(PEFT) 기술을 사용한 사전학습된 모델의 식별자 정의
config = PeftConfig.from_pretrained(peft_model_id) #perfit_model_id로 설정된 PEFT 설정을 사용해서 모델에 대한 구성 객체를 초기화함. 모델 아키텍처와 미세 조정 세부 정보를 위한 관련 설정과 매개변수 로드
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, # 모델을 4비트 정밀도로 로드하도록 지정, 메모리 사용량 줄임.
    bnb_4bit_use_double_quant=True, # 이중 양자화를 활성화하여 정밀도 손실을 줄이는 데 도움
    bnb_4bit_quant_type="nf4", # nf4라는 특정 양자화 형식을 사용하도록 지정
    bnb_4bit_compute_dtype=torch.bfloat16 #계산 데이터 형식을 bfloat16으로 설정, 메모리 사용량 줄이면서 성능 유지하는 데 도움.
)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, quantization_config=bnb_config, device_map={"":0})

# 사전학습된 모델 config.base_model_name_or_path로부터 인과언어모델링을 위한 기본 모델 초기화
# 모델은 지정된 양자화 설정(bnb_config)으로 로드되고 디바이스맵에 할당된다.
# device_map={"":0}: 첫번째 GPU에 할당

model = PeftModel.from_pretrained(model, peft_model_id)

# 사전학습된 PEFT 모델을 사용해서 기본 모델(model)을 PEFT로 감쌈.
# peft 모델에서 학습된 추가 미세 조정 매개변수와 기술을 기본 모델에 통합해서 모델이 특수한 훈련을 활용할 수 있도록 한다.

tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# 사전학습된 config.base_model_name_or_path에 해당하는 토크나이저를 초기화
# 토크나이저 : 텍스트를 모델이 처리할 수 있는 토큰 ID로 변환하고 모델의 출력을 다시 텍스트로 변환한다.

model.eval()
# 모델을 평가 모드로 설정.

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(30080, 5120)
        (emb_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-39): 40 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_dropout): Dropout(p=0.0, inplace=False)
            (post_mlp_dropout): Dropout(p=0.0, inplace=False)
            (attention): GPTNeoXSdpaAttention(
              (rotary_emb): GPTNeoXRotaryEmbedding()
              (query_key_value): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=5120, out_features=15360, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
            

next token generation

In [19]:

prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'
# 생성할 텍스트의 시작점으로 사용될 입력 문장 정의.

with torch.no_grad(): # 자동 미분 비활성화 _ 모델을 평가 모드로 사용해서 메모리 절약, 계산속도 높임
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  # 입력 문장을 토크나이저로 토큰화해서 토큰 ID로 변환, Pytorch 텐서로 반환
  # non-blocking=True : GPU와 CPU 간의 데이터 전송이 비동기적으로 이루어져서 처리속도가 빨라지도록 함.

  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=64)
  #모델 사용해서 텍스트 생성
  # do_sample : 확률적으로 다음 단어를 선택해서 다양한 텍스트 출력을 생성하도록 한다.
  # temp : 샘플링의 랜덤성 조절, 값이 높을수록 더 무작위
  # max length : 생성할 최대 토큰 수


  generated = tokenizer.batch_decode(gen_tokens)[0]
  # 생성된 토큰 ID 배열(gen_tokens)을 원래의 텍스트로 디코딩해서 사람이 읽을 수 있는 문자열로 변환
  # batch_decode : 여러 샘플을 동시에 디코딩 할 수 있지만 우리는 단일 샘플만 생성하므로 첫번째 결과[0]만 추출.

print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 문제를 해결하고,또다른 차원의 삶을 열어갈 수 있다는 것이다.<|endoftext|>


# QA

In [20]:
def gen(x):
    q = f"### 질문: {x}\n\n### 답변:"
    # print(q)
    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt', #q를 토큰화하고 파이토치 텐서로 반환
            return_token_type_ids=False #토큰 타입 id 반환 x하도록 설정
        ).to('cuda'),
        max_new_tokens=50,
        early_stopping=True, #모델이 종료 토큰id(eos_token_id)를 생성하면 조기 종료. 더이상 토큰 생성x
        do_sample=True, #확률적으로 다음 토큰을 샘플링해서 출력을 다채롭게
        eos_token_id=2, #종료 토큰 id를 2로 설정.
    )
    print(tokenizer.decode(gened[0]))

In [21]:
gen('건강하게 살기 위한 세 가지 방법은?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 건강하게 살기 위한 세 가지 방법은?

### 답변: (1) 몸을 움직이는 방법. 움직이지 않고 지내면 살찌기 쉽다. (2) 걷거나 달리거나 몸을 움직이는 기회를 많이 갖는 방법. (3) 생활습관의 개선. 즉 적당히 움직이


# max_new_tokens = 100으로 변화.

In [32]:
def gen(x):
    q = f"### 질문: {x}\n\n### 답변:"
    # print(q)
    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt', #q를 토큰화하고 파이토치 텐서로 반환
            return_token_type_ids=False #토큰 타입 id 반환 x하도록 설정
        ).to('cuda'),
        max_new_tokens=100,
        early_stopping=True, #모델이 종료 토큰id(eos_token_id)를 생성하면 조기 종료. 더이상 토큰 생성x
        do_sample=True, #확률적으로 다음 토큰을 샘플링해서 출력을 다채롭게
        eos_token_id=2, #종료 토큰 id를 2로 설정.
    )
    print(tokenizer.decode(gened[0]))

In [33]:
gen('건강하게 살기 위한 세 가지 방법은?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 건강하게 살기 위한 세 가지 방법은?

### 답변: 안녕하세요. 대구피부과추천 세브란스피부과 입니다. 오늘 대구피부과추천 세브란스피부과에서 건강하게 살기위한 건강법  에 대해서 알려드리도록 하겠습니다. 먼저 세가지 건강법이란 첫번째는 올바른 식사법을 말합니다. 두번째는 적절한 운동법 세번째는 적절한 휴식법을 들 수 있습니다. 오늘 대구피부과추천 세브란스피부과에서 말하는 건강하게


# top k 변화
- 텍스트 생성 과정에서 사용되는 샘플링 기법 중 하나
- 모델이 다음 토큰을 선택할 때 사용할 수 있는 후보 토큰의 개수를 제한하는 역할을 한다.

- topk = 50 : 가장 확률이 높은 50개의 토큰 중에서 다음 토큰을 샘플링한다. 모델은 50개의 후보군 중에서 랜덤하게 토큰을 선택.
생성과정에서 확률이 높은 상위 50개의 토큰만 선택가능하고 이는 생성된 텍스트가 더 신뢰성 있게 한다.  

In [22]:
prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=64, top_k=50)
  generated = tokenizer.batch_decode(gen_tokens)[0]

print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 어려운 난제들을 해결하고 있다.​​​​<|endoftext|>


In [35]:
prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=64, top_k=3)
  generated = tokenizer.batch_decode(gen_tokens)[0]

print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 '의식'이라는 문제를 해결하고자 한다. ​​그렇다면 과연 미래에는 인간의 의식에 어떤 변화가 일어날까?​​1. 인류는 두뇌의 한계


In [36]:
prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=64, top_k=1)
  generated = tokenizer.batch_decode(gen_tokens)[0]

print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 문제들을 해결할 수 있을 것으로 기대하고 있다.​​​​​​​​​​​​​​


In [38]:
prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=64, top_k=2)
  generated = tokenizer.batch_decode(gen_tokens)[0]

print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 문제를 해결하는 것을 목표로 하고 있다.​​​​​​​​​​​​​​​
